dpownload and compile optical flow 

not nessasery if you run in colab

In [ ]:
!git clone https://github.com/pathak22/pyflow.git

In [ ]:
cd 'pyflow/'

In [ ]:
!python setup.py build_ext -i
!python demo.py 
cd ..

In [ ]:
!pip install plyfile

ISFM flow

In [ ]:
import os
import numpy as np
from pyflow import pyflow
import cv2
from sklearn.neighbors import KDTree

def pixels_matching_selection(img_p, img_n):
    alpha = 0.012
    ratio = 0.75
    minWidth = 20
    nOuterFPIterations = 7
    nInnerFPIterations = 1
    nSORIterations = 30
    colType = 0

    vx, vy, im2W = pyflow.coarse2fine_flow(img_p, img_n, alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
                                           nSORIterations, colType)

    img_size = img_p.shape[:2]
    xpast, ypast = np.meshgrid(range(0, img_size[1]), range(0, img_size[0]))
    xnext = xpast + vx
    ynext = ypast + vy

    x1 = xpast[(xnext > 0) * (xnext < img_size[1]) * (ynext > 0) * (ynext < img_size[0])]
    y1 = ypast[(xnext > 0) * (xnext < img_size[1]) * (ynext > 0) * (ynext < img_size[0])]
    x2 = xnext[(xnext > 0) * (xnext < img_size[1]) * (ynext > 0) * (ynext < img_size[0])]
    y2 = ynext[(xnext > 0) * (xnext < img_size[1]) * (ynext > 0) * (ynext < img_size[0])]

    p1 = np.column_stack((x1, y1))
    p2 = np.column_stack((x2, y2))

    p_past = np.column_stack((xpast.ravel(), ypast.ravel()))
    p_next = np.column_stack((xnext.ravel(), ynext.ravel()))

    return p1, p2

def reproj_error(K, Rt_cur, x_3d, x_2d):
    ret = cv2.Rodrigues(Rt_cur[:, :3])
    x_proj = cv2.projectPoints(x_3d.T, ret[0], Rt_cur[:, 3], K, None)
    x_proj = x_proj[0][:, 0, :]
    error = np.sum((x_proj - x_2d) ** 2) / x_proj.shape[0]
    # error_pix = (x_proj[:, 0] - x_2d[:, 0]) ** 2 + (x_proj[:, 1] - x_2d[:, 1]) ** 2
    return error

def save_ply(X, texture, name):
    from plyfile import PlyData, PlyElement

    # error, _, _ = reproj_error(K, Rt_pnp, X, p2_cur)
    X_mean = np.mean(X, axis=1)
    X_std = np.std(X, axis=1)
    inds = X[2, :] > 0
    if (X[2, :] < 0).any():
        print(X_mean)
        print(X_std)
        X[2, X[2, :] > (X_mean[2] + X_std[2]) * 0.5] = (X_mean[2] + X_std[2] * 0.25)

        X[1, X[1, :] > (X_mean[1] + X_std[1]) * 0.5] = (X_mean[1] + X_std[1] * 0.25)
        X[1, X[1, :] < (X_mean[1] - X_std[1]) * 0.5] = (X_mean[1] - X_std[1] * 0.25)

        X[0, X[0, :] > (X_mean[0] + X_std[0]) * 0.5] = (X_mean[0] + X_std[0] * 0.25)
        X[0, X[0, :] < (X_mean[0] - X_std[0]) * 0.5] = (X_mean[0] - X_std[0] * 0.25)

        print(X[0, inds].max(), X[1, inds].max(), X[2, inds].max())
        print(X[0, inds].min(), X[1, inds].min(), X[2, inds].min())
    X = X[:, inds].T
    texture = texture[inds, :]

    vert_list = []
    for vert, rgb in zip(X, texture):
        vert_list.append((vert[0], vert[1], vert[2], rgb[0], rgb[1], rgb[2]))
    vertex = np.array(vert_list, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('red', 'u1'), ('green', 'u1'),
                                        ('blue', 'u1')])

    el = PlyElement.describe(vertex, 'vertex')
    PlyData([el], text=True).write(name + '.ply')

def isfm0(p1_cur, p2_cur, K):
    pose_cur = cv2.recoverPose(p1_cur.astype('float32').T, p2_cur.astype('float32').T, K, np.zeros(5), K,
                               np.zeros(5))
    Rt_cur = np.hstack((pose_cur[2], pose_cur[3]))
    match_cur = pose_cur[4]
    p1_cur = p1_cur[np.where(match_cur)[0], :]
    p2_cur = p2_cur[np.where(match_cur)[0], :]
    P0 = K.dot(np.column_stack((np.eye(3), np.zeros(3))))
    P1 = K.dot(Rt_cur)
    X_cur = cv2.triangulatePoints(P0, P1, p1_cur.astype('float32').T, p2_cur.astype('float32').T)
    X_cur = X_cur[0:3, :] / X_cur[3, :]
    return p2_cur, Rt_cur, X_cur

def isfm(p2_prev, p1_cur, p2_cur, X_prev, K):

  kdt = KDTree(p2_prev)
  kdt_res = kdt.query(p1_cur)
  max_pix_dist = 1
  mask1 = kdt_res[1][kdt_res[0] < max_pix_dist]
  mask2 = (kdt_res[0] < max_pix_dist).nonzero()[0]

  pnp = cv2.solvePnPRansac(X_prev[:, mask1].astype('float32').T,
                            p2_cur[mask2, :].astype('float32'), K, None,
                            iterationsCount=1000, reprojectionError=0.1)
  R = cv2.Rodrigues(pnp[1])[0]
  t = pnp[2]

  Rt_pnp = np.hstack((R, t))
  P0 = K.dot(Rt_prev)
  P1 = K.dot(Rt_pnp)
  X_pnp = cv2.triangulatePoints(P0, P1, p1_cur.astype('float32').T,
                                p2_cur.astype('float32').T)
  X_pnp = X_pnp[0:3, :] / X_pnp[3, :]

  return X_pnp, Rt_pnp

#### define intrinsic camera metrix (not precise)
f = 1702
img = cv2.imread('frames/frame0.png')[:, :, ::-1]
K = np.array([[f, 0, (img.shape[1] - 1) / 2],
              [0, f, (img.shape[0] - 1) / 2], 
               [0, 0, 1]])

track = np.zeros((1, 3)) ## final camera path
### create folder for models at each reconstruction step
if os.path.isdir('models') == False:
    os.mkdir('models')

skip = 5 ## skip frames for paralax stability
np.random.seed(100) ### very important for RANSAC initializatin !

#### main reconstruction loop
for k in range(0, 100 * skip, skip):
    print('frame # ' + str(k))

### pairwise frame matching by optical flow
    texture = cv2.imread('frames/frame' + str(k) + '.png')[:, :, ::-1]
    img_1 = texture.astype(float) / 255.
    img_2 = cv2.imread('frames/frame' + str(k + skip) + '.png')[:, :, ::-1]
    img_2 = img_2.astype(float) / 255.
    p1_cur, p2_cur = pixels_matching_selection(img_1, img_2)

    if k == 0: ### initial model reconstruction: the world coordinates system 
    #origin is located at the first frame
        p2_cur, Rt_cur, X_cur = isfm0(p1_cur, p2_cur, K)

        p2_prev = np.copy(p2_cur)
        X_prev = np.copy(X_cur)
        Rt_prev = np.copy(Rt_cur)

        C = -Rt_cur[:, :3].T.dot(Rt_cur[:, 3])## camera position in space
        track = np.vstack((track, C.reshape(1, -1)))
        
    else:       

        X_pnp, Rt_pnp = isfm(p2_prev, p1_cur, p2_cur, X_prev, K)

        X_prev = np.copy(X_pnp)
        p2_prev = np.copy(p2_cur)
        Rt_prev = np.copy(Rt_pnp)

        C = -Rt_prev[:, :3].T.dot(Rt_prev[:, 3]) ## camera position in space
        track = np.vstack((track, C.reshape(1, -1)))

## reconstraction quality error / model reprojection error
        error = reproj_error(K, Rt_pnp, X_pnp, p2_cur) 
        if error < 2: ## save only qualitative models with reprojection error 
        #up 2 pixels (optional)
          save_ply(X_pnp, texture[p1_cur[:, 1], p1_cur[:, 0], :],
                  'models/X_' + str(k) + '_' + str(np.round(error, 2)))  
            
### save camera movement path
np.savetxt('models/track.txt', track)